In [13]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import TransformationPass
import random
from random import choices
import itertools
from qiskit_aer import Aer
from qiskit import ClassicalRegister
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [3]:
nqubits = 2
nshots = 20000         

def getParity(n):
    parity = 0
    while n:
        parity = ~parity
        n = n & (n-1)
    return parity

def expectation_value_from_counts(counts):
    exp_val = 0
    for x in map(''.join, itertools.product('01', repeat=nqubits)):
        if x in counts:
            if getParity(int(x,2)) == -1:
                exp_val -= counts[x]
            if getParity(int(x,2)) == 0:
                exp_val += counts[x]
    return exp_val / nshots

clifford_exponents = np.array([0.0, 0.5, 1.0, 1.5])
clifford_angles = [exponent * np.pi for exponent in clifford_exponents]

class RZTranslator(TransformationPass):
    def run(self, dag):
        N = 0.8  # TUNABLE: replace 30% of the non-Clifford gates
        for node in dag.op_nodes():
            if node.op.name in ["rz"]:
                angle = node.op.params[0]
                replacement = QuantumCircuit(1)
                if node.op.name == "rz":
                    if angle not in clifford_angles:
                        if choices([0,1], [N, 1-N])[0] == 0:
                            replacement.rz(random.choice(clifford_angles), 0)
                        else:
                            replacement.rz(angle, 0)
                dag.substitute_node_with_dag(node, circuit_to_dag(replacement))
        return dag

def append_gates(qc):
    for rep in range(5):
        for qubit in range(nqubits):
            qc.h(qubit)  
        for qubit in range(nqubits)[::2]:
            qc.rz(1.75, qubit)
        for qubit in range(nqubits)[1::2]:
            qc.rz(2.31, qubit)
        for qubit in range(nqubits)[::2]:
            qc.cx(qubit, qubit + 1)     
        for qubit in range(nqubits)[::2]:
            qc.rz(-1.17, qubit)
        for qubit in range(nqubits)[1::2]:
            qc.rz(3.23, qubit)
        for qubit in range(nqubits):
            qc.rx(np.pi / 2, qubit)    
    return qc

In [5]:
qc = QuantumCircuit(nqubits)
append_gates(qc)
qc.draw("mpl")
training_circuits_no_measurement_all = []
for _ in range(50):
    training_circuits_no_measurement_all.append(RZTranslator()(qc))
print(*training_circuits_no_measurement_all)

     ┌───┐┌───────┐     ┌───────────┐┌─────────┐┌───┐ ┌───────┐       »
q_0: ┤ H ├┤ Rz(0) ├──■──┤ Rz(-1.17) ├┤ Rx(π/2) ├┤ H ├─┤ Rz(0) ├────■──»
     ├───┤├───────┤┌─┴─┐└┬──────────┤├─────────┤├───┤┌┴───────┴─┐┌─┴─┐»
q_1: ┤ H ├┤ Rz(π) ├┤ X ├─┤ Rz(3π/2) ├┤ Rx(π/2) ├┤ H ├┤ Rz(3π/2) ├┤ X ├»
     └───┘└───────┘└───┘ └──────────┘└─────────┘└───┘└──────────┘└───┘»
«      ┌───────┐  ┌─────────┐┌───┐ ┌───────┐       ┌───────────┐┌─────────┐»
«q_0: ─┤ Rz(π) ├──┤ Rx(π/2) ├┤ H ├─┤ Rz(π) ├────■──┤ Rz(-1.17) ├┤ Rx(π/2) ├»
«     ┌┴───────┴─┐├─────────┤├───┤┌┴───────┴─┐┌─┴─┐└┬──────────┤├─────────┤»
«q_1: ┤ Rz(3π/2) ├┤ Rx(π/2) ├┤ H ├┤ Rz(3π/2) ├┤ X ├─┤ Rz(3π/2) ├┤ Rx(π/2) ├»
«     └──────────┘└─────────┘└───┘└──────────┘└───┘ └──────────┘└─────────┘»
«     ┌───┐┌───────┐     ┌──────────┐┌─────────┐┌───┐ ┌───────┐       »
«q_0: ┤ H ├┤ Rz(π) ├──■──┤ Rz(3π/2) ├┤ Rx(π/2) ├┤ H ├─┤ Rz(π) ├────■──»
«     ├───┤├───────┤┌─┴─┐├──────────┤├─────────┤├───┤┌┴───────┴─┐┌─┴─┐»
«q_1: ┤ H ├┤ Rz(0) ├┤ X ├┤ Rz(3.23) ├┤ 

In [6]:
psi_qc = qc
observable = SparsePauliOp.from_list([('ZZ', 1)])
estimator = Estimator()
result = estimator.run([psi_qc], [observable]).result()
exp_val_qc = result.values[0].real
print("Exact expectation value of circuit of interest:", exp_val_qc)

Exact expectation value of circuit of interest: 0.20422260124463706


In [7]:
noiseless_exp_vals = []
training_circuits_no_measurement = []
exp_val_qc_real = exp_val_qc

for circuit in training_circuits_no_measurement_all:
    result = estimator.run([circuit], [observable]).result()
    expectation_value = result.values[0].real
    if exp_val_qc_real - 0.05 <= expectation_value <= exp_val_qc_real + 0.05:
        noiseless_exp_vals.append(expectation_value)
        training_circuits_no_measurement.append(circuit)

for exp_val in noiseless_exp_vals:
    print('Exact expectation value:', exp_val)

Exact expectation value: 0.20021171029574675
Exact expectation value: 0.17824605564949197
Exact expectation value: 0.19541387091345536


In [8]:
qr_qc = QuantumRegister(nqubits)
cr_qc = ClassicalRegister(nqubits)
circ_qc = QuantumCircuit(qr_qc, cr_qc)
circ_qc.append(qc.to_instruction(), [qubit for qubit in range(nqubits)])
circ_qc.measure(qr_qc, cr_qc)

In [9]:
training_circuits_with_measurement = []
for circuit in training_circuits_no_measurement:
    qr = QuantumRegister(nqubits)
    cr = ClassicalRegister(nqubits)
    circ = QuantumCircuit(qr, cr)
    circ.append(circuit.to_instruction(), [qubit for qubit in range(nqubits)])
    circ.measure(qr, cr)
    training_circuits_with_measurement.append(circ)

print(*training_circuits_with_measurement)

      ┌─────────────────┐┌─┐   
q2_0: ┤0                ├┤M├───
      │  circuit-105167 │└╥┘┌─┐
q2_1: ┤1                ├─╫─┤M├
      └─────────────────┘ ║ └╥┘
c1: 2/════════════════════╩══╩═
                          0  1        ┌─────────────────┐┌─┐   
q3_0: ┤0                ├┤M├───
      │  circuit-105167 │└╥┘┌─┐
q3_1: ┤1                ├─╫─┤M├
      └─────────────────┘ ║ └╥┘
c2: 2/════════════════════╩══╩═
                          0  1        ┌─────────────────┐┌─┐   
q4_0: ┤0                ├┤M├───
      │  circuit-105167 │└╥┘┌─┐
q4_1: ┤1                ├─╫─┤M├
      └─────────────────┘ ║ └╥┘
c3: 2/════════════════════╩══╩═
                          0  1 


In [10]:
device_backend = FakeVigoV2() # 
coupling_map = device_backend.configuration().coupling_map
sim_vigo = AerSimulator.from_backend(device_backend)

In [14]:
noisy_exp_vals = []
for circuit in training_circuits_with_measurement:
    tqc = transpile(circuit, sim_vigo)
    result_noise = sim_vigo.run(tqc, shots=nshots).result()
    counts_noise = result_noise.get_counts(0)
    noisy_exp_val = expectation_value_from_counts(counts_noise)
    print("Noisy expectation value from counts:", noisy_exp_val)
    noisy_exp_vals.append(noisy_exp_val)

Noisy expectation value from counts: 0.1259
Noisy expectation value from counts: 0.124
Noisy expectation value from counts: 0.141


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Örnek veri
noisy_exp_vals = torch.tensor(noisy_exp_vals, dtype=torch.float32)  # Girdi verileri
noiseless_exp_vals = torch.tensor(noiseless_exp_vals, dtype=torch.float32)  # Hedef veriler

# Neural Network tanımı
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Model, Loss ve Optimizer tanımı
input_dim = noisy_exp_vals.shape[0]
output_dim = noiseless_exp_vals.shape[0]
model = NeuralNetwork(input_dim, hidden_dim=32, output_dim=output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Eğitim döngüsü
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(noisy_exp_vals)
    loss = criterion(outputs, noiseless_exp_vals)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

print("Eğitim tamamlandı.")


C:\Users\Özgür Nazlı\AppData\Local\Temp\ipykernel_16612\2567832024.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  noisy_exp_vals = torch.tensor(noisy_exp_vals, dtype=torch.float32)  # Girdi verileri
C:\Users\Özgür Nazlı\AppData\Local\Temp\ipykernel_16612\2567832024.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  noiseless_exp_vals = torch.tensor(noiseless_exp_vals, dtype=torch.float32)  # Hedef veriler


Epoch [0/1000], Loss: 0.0310
Epoch [100/1000], Loss: 0.0000
Epoch [200/1000], Loss: 0.0000
Epoch [300/1000], Loss: 0.0000
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Eğitim tamamlandı.


In [ ]:
# Plotting
import matplotlib.pyplot as plt
plt.plot(noisy_exp_vals, noiseless_exp_vals, 'o', label='Original data')
x_pred = np.linspace(min(noisy_exp_vals), max(noisy_exp_vals), 1000).reshape(-1, 1)
y_pred, sigma = gp.predict(x_pred, return_std=True)
plt.plot(x_pred, y_pred, 'r', label='Gaussian fit')
plt.fill_between(x_pred.ravel(), y_pred.ravel() - sigma, y_pred.ravel() + sigma, alpha=0.2, label='Uncertainty')
plt.legend()
plt.show()